## Проект. Невероятно, но факт!

![](data/math02_01.png)

"Притормози, притормози...". Недавно вы завершили модуль по теории вероятности и пришло время закрепить полученные знания и навыки - давайте попробуем смоделировать процесс работы сервиса такси. Как полагается, у нас есть водители, которым поступают заказы. Процесс приема заказа хорошо моделируется распределением Пуассона - которое позволяет оценить интенсивность событий, или среднее количество событий за единицу времени.

При решении этого проекта вы будете моделировать данные из заданного распределения, подберёте параметры распределения, описывающего процесс, примените найденную модель для ответа на практические вопросы владельцев сервиса.

## Задание 1. Моделирование пуассоновской случайной величины

Сервис по заказу такси “Блиц” расширяется и открывается в городе Барнаул. Необходимо рассчитать примерную нагрузку на таксистов. Доступа к данным с заказами у нас нет – ещё бы, это же ценная информация! Поэтому придется эти данные сгенерировать. Напишем функцию, возвращающую случайные значения, распределенные по закону Пуассона.

Как вы знаете, вероятность того, что пуассоновская случайная величина имеет значение $k$, равно

$f(k,l) = l^k*e^-l / (k!)$, где $l$ (лямбда) - параметр распределения.

На практике мы немного модифицируем эту формулу, чтобы оптимизировать программу: при решении подсчитывать факториал на каждой итерации не нужно.

Для того, чтобы сгенерировать значения такой случайной величины, воспользуемся следующим алгоритмом: выберем случайное $y$ число из промежутка [0, 1]. Затем будем суммировать $f(k,l)$ до тех пор, пока сумма не превысит выбранного числа $y$. Тот $k$, на котором сумма превысила $y$ - это и есть наш результат.

Таким образом мы находим, при каком $k$ значение кумулятивной функции распределения превышает выбранное случайное число.

- **Входные данные:** параметр `l`.
- **Результат:** напишите функцию `poisson(l)`, которая будет возвращать значения, распределённые по закону Пуассона с параметром `l`. Например, `poisson(3)` чаще всего будет возвращать 2 или 3. 

In [1]:
import numpy as np
import random
import math

In [2]:
def f(k, l):
    return (math.pow(l, k) * math.exp(-l)) / math.factorial(k)

In [3]:
def poisson(l):
    y = random.uniform(0, 1)
    k = 0
    total = 0
    while total <= y:
        total += f(k, l)
        k += 1
    return k - 1

In [4]:
def poisson(l):
    e = np.e ** -l
    y = np.random.uniform()
    k = 0
    fact = 1
    total = 0
    while total <= y:
        f = l**k * e / fact
        total += f
        k += 1
        fact = fact * k
    return k - 1

In [5]:
poisson(3)

5

## Задание 2. Насколько модельные данные отличаются от реальных?

Отлично, сервис “Блиц” зашел на рынок транспортных услуг Барнаула и успешно доставляет пассажиров из точки А в точку N! Пришло время проверить, насколько сгенерированные нами ранее данные отличаются от реальных.

Для этого напишем функцию, которая генерирует массив случайных значений и сравнивает его с реальными данными – находит средний квадрат разности. Это стандартная метрика для нахождения отклонения одной величины от другой.

Будем считать, что у нас есть данные по дням за последний год (**365** чисел). А именно, пусть у нас есть массив, $i$-й элемент которого содержит число пассажиров, перевезённых одним водителем в $i$-й день года. Будем считать, что эта случайная величина имеет **распределение Пуассона**.

Вам требуется найти параметр _lambda_ (напомним, что среднее значение пуассоновской величины как раз равно этому параметру). Затем, используя результат из предыдущей задачи, сгенерируйте 365 чисел. Имея два массива (исходные данные и сгенерированные вами), посчитайте средний квадрат разности между соответствующими значениями: `sum((data_real[i] - data_generated[i])**2)/365`.

- **Входные данные:** массив `data_real` из 365 элементов, каждый из элементов равен количеству перевезённых пассажиров в соответствующий день.
- **Результат:** напишите функцию `poisson_error(data)`, которая сгенерирует массив `data_generated` и вернёт средний квадрат разности между исходными и сгенерированными данными.

In [6]:
def poisson_error(data):
    lambda_param = sum(data) / len(data)
    fake = [poisson(lambda_param) for x in data]
    e = sum([(data[i] - fake[i])**2 for i, x in enumerate(data)]) / len(data)
    
    return e

In [7]:
real = [poisson(10) for i in range(365)]
poisson_error(real)

20.942465753424656

## Задание 3. Вероятность перевезти ровно k пассажиров

Корбен, водитель сервиса такси “Блиц” решил заключить пари с другом. Он утверждает, что завтра у него будет ровно $k$ заказов. Используя исторические данные, найдите параметр пуассоновского распределения $l$ (лямбда) и оцените эту вероятность.

- **Входные данные:** данные по поездкам `data` за предыдущие 365 дней и число `k`.
- **Результат:** напишите функцию `poisson_prob(data, k)`, вероятность того, что водитель завтра перевезёт ровно `k` пассажиров.
- **Пример:** допустим, что `data` имеет распределение Пуассона с параметром `l=3`. Тогда функция `poisson_prob(data, 3)` должна вернуть `0.22404180765538773`. 

In [8]:
def poisson_prob(data, k):
    l = sum(data) / len(data)
    return f(k, l)

In [9]:
real = [poisson(3) for i in range(365)]
poisson_prob(real, 3)

0.2236055264748397

## Задание 4. Время ожидания следующего пассажира

Пусть заказы в течение рабочей смены (**8 часов**) водителя не имеют пиков (т. е., это процесс Пуассона). Корбен хочет сделать перерыв на обед, но не хочет пропускать заказы - тут без вашей помощи не обойтись. Посчитайте матожидание времени до следующего заказа, чтобы Корбен смог спланировать свой перекус.

- **Входные данные:** как и в предыдущей задаче, вам доступны исторические данные по заказам за предыдущий год `data` (массив `data` из 365 элементов).
- **Результат:** напишите функцию `time_to_order(data)`, которая восстанавливает параметр `l` пуассоновского распределения и возвращает ожидаемое время до следующего заказа в часах.
- **Пример:** если вычисленный параметр распределения `l` будет равен 3, то функция `time_to_order` должна вернуть 2.6666666666666665. Подсказка: используйте экспоненциальное распределение.

In [10]:
def time_to_order(data):
    work_day = 8 #hours
    l = sum(data) / len(data)
    return work_day / l

In [11]:
real = [poisson(3) for i in range(365)]
time_to_order(real)

2.7625354777672655

## Задание 5. Смесь распределений

Количество клиентов сервиса “Блиц” разнится изо дня в день. В целом, мы можем выделить обычные дни (основной поток), а также дни, когда происходят некоторые “знаковые” события (дополнительный поток): например, концерты или футбольные матчи. В такие “знаковые” дни пассажиропоток, а следовательно и число клиентов, растет. Необходимо найти параметры распределений основного потока и дополнительного.

Пусть вам даны количества поездок за предыдущие 365 дней и массив с номерами дней, которые были праздничными. Необходимо вернуть параметры пуассоновского распределения для обычных дней и для праздничных.

- **Входные данные:** массив `data` с данными о количестве поездок и массив `days` с номерами праздничных дней (индексация с нуля).

- **Результат:** напишите функцию `estimate_parameters(data, days)`, возвращающую кортеж из двух чисел _(l_usual, l_special)_. *l_usual* - параметр распределения в обычные дни, *l_special* - в праздничные. 

In [12]:
def estimate_parameters(data, days):
    usual = [data[i] for i in data if i not in days]
    special = [data[i] for i in days]
    l_usual = sum(usual) / len(usual)
    l_special = sum(special) / len(special)
    return l_usual, l_special

In [13]:
days = list(set([random.randint(0, 365) for i in range(50)]))
real = [
    poisson(10 if i in days else 3)
    for i in range(365)
]
estimate_parameters(real, days)

(2.8444444444444446, 10.325581395348838)